# Library Agent Creation

Notebook below connects to an existing Azure AI Foundry workspace and instantiates an agent which supports question/answering over Goodreads book data (descriptions & reviews) retrieved from both Azure AI Search and an Azure SQL database

### Install missing dependencies

In [12]:
# ! pip install -r ../../requirements.txt

### Import required packages

In [13]:
import json

import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import MessageTextContent
from azure.ai.projects.models import CodeInterpreterTool, MessageAttachment
from azure.ai.projects.models import FilePurpose
from pathlib import Path
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
import base64

from azure.ai.projects.models import (
    AgentEventHandler,
    FunctionTool,
    MessageDeltaChunk,
    RequiredFunctionToolCall,
    RunStep,
    RunStepStatus,
    SubmitToolOutputsAction,
    ThreadMessage,
    ThreadRun,
    ToolOutput,
)
from typing import Any
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

True

### Load schema for target search index

In [14]:
with open('../schema/ai_search_schema.json', 'r') as f:
    ai_search_schema = f.read()

json.loads(ai_search_schema)

{'id': 'A unique identifier for the book in the index.',
 'sourcefileref': 'A reference to the original file source, which can be null if not applicable.',
 'Name': 'The title of the book.',
 'PublishMonth': 'The month the book was published.',
 'PublishDay': 'The day of the month the book was published.',
 'Publisher': 'The name of the entity that published the book.',
 'PublishYear': 'The year the book was published.',
 'Language': 'The language in which the book is written, represented in a string format.',
 'Authors': 'The name(s) of the author(s) of the book.',
 'ISBN': 'The International Standard Book Number assigned to the book.',
 'Description': "A brief summary or description of the book's content.",
 'embeddings': "A list of numerical values representing the book's content in a vector space, often used for similarity searches or machine learning applications."}

### Load schema for target SQL database

In [15]:
with open('../schema/sql_table_schema.json', 'r') as f:
    sql_schema = f.read()

json.loads(sql_schema)

{'dbo.goodreads_book_data': {'Name': {'type': 'nvarchar',
   'description': 'The title of the book.'},
  'ISBN': {'type': 'nvarchar',
   'description': 'The International Standard Book Number assigned to the book.'},
  'RatingDist1': {'type': 'bigint',
   'description': 'The number of 1-star reviews the book has received.'},
  'RatingDist2': {'type': 'bigint',
   'description': 'The number of 2-star reviews the book has received.'},
  'RatingDist3': {'type': 'bigint',
   'description': 'The number of 3-star reviews the book has received.'},
  'RatingDist4': {'type': 'bigint',
   'description': 'The number of 4-star reviews the book has received.'},
  'RatingDist5': {'type': 'bigint',
   'description': 'The number of 5-star reviews the book has received.'},
  'RatingDistTotal': {'type': 'bigint',
   'description': 'The total number of reviews the book has received across all star ratings.'},
  'CountsOfReview': {'type': 'bigint',
   'description': 'The total count of reviews the book ha

### Import utility function definitions 

In [16]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), '../utility_functions'))
sys.path.append(r'C:\Users\nikwieci\Documents\Engagements\Eaton\Eaton_Agent_Streaming_Sample\src\utility_functions')

from agent_functions import agent_functions
functions = FunctionTool(functions=agent_functions)

### Create connection to Azure AI Foundry

In [17]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ['AZURE_AI_FOUNDRY_CONNECTION_STRING'],
)

### Define and Create Agent

Note: after creation, you can reference this agent by ID in your FastAPI app

In [18]:
agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="librarian-agent",
    instructions=f"""You are a helpful librarian assisting users with questions about Goodreads book data stored in multiple sources: **an Azure AI Search Index**, **a SQL database**, and you also have access to a **code interpreter** for data analysis and visualization. You have the ability to retrieve and analyze data from these sources to provide factual, grounded responses.

1. **Tools and Their Uses**  
   - **Azure AI Search Index**: Best for searching by titles, descriptions, authors, publishers, or keywords to find relevant books.  
   - **SQL Database**: Contains the same core data (linked by ISBN) as the search index, plus detailed review statistics (e.g., average rating, count of reviews per star rating). Use SQL queries to retrieve review-based details.  
   - **Code Interpreter**: Use this tool for creating visualizations, performing data analysis, or other code-based tasks that could help in answering users’ questions (e.g., charting ratings over time).

2. **Search and Retrieval**  
   - Always start by checking whether a search index query or a SQL query (or both) can provide the necessary information.  
   - You may perform multiple queries (search or SQL) and use the code interpreter if needed to clarify or enhance your findings.

3. **Grounding and Citations**  
   - Base your answers **only** on the facts found in the retrieved data.  
   - For citations, use the book’s **ISBN** in square brackets, for example: `[ISBN1]`.  
   - If you cite multiple sources, list them separately (e.g., `[ISBN1][ISBN2]`).  
   - If your initial attempts do not yield sufficient information, try different queries or clarifications as needed.

4. **Handling Insufficient Information**  
   - If, after attempting relevant searches or queries, you still cannot find the information, politely inform the user that the data is not available.

5. **Communication Style**  
   - Be friendly, respectful, and supportive, as a librarian would be.  
   - If the user’s question is not in English, answer in the same language.  
   - If you need more details from the user to find the right information, ask clarifying questions.

6. **Constraints**  
   - Do not generate or add content that is not supported by the results from your tools.  
   - Refrain from providing speculation or extraneous details that are not directly supported by the data.

---

### AI Search Index Schema: {ai_search_schema}

---

### SQL Table Schema: {sql_schema}
        
        """,
    temperature=0.0,
    tools=functions.definitions + CodeInterpreterTool().definitions,
)
agent_id = agent.id
print(f"Agent ID: {agent_id}")

Agent ID: asst_Grkk9RtOB87PTtYSaRWXHCNp
